In [46]:
#ライブラリ
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
import optuna
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

#検証方法：クロスバリデーション
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

#モデル
import lightgbm as lgb

In [29]:
#データ準備
train =pd.read_table('data/train.tsv', index_col=0)
test = pd.read_table('data/test.tsv',index_col=0)
sample = pd.read_table('data/sample_submit.tsv' , index_col=0, header=None)

In [30]:
train= train.reset_index(drop=True)
train

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,1,female,35.0,1,0,53.1000,S
1,0,3,male,35.0,0,0,8.0500,S
2,0,3,male,2.0,3,1,21.0750,S
3,1,2,female,14.0,1,0,30.0708,C
4,1,1,female,58.0,0,0,26.5500,S
...,...,...,...,...,...,...,...,...
440,0,3,male,47.0,0,0,9.0000,S
441,1,2,female,28.0,1,0,24.0000,C
442,1,1,female,56.0,0,1,83.1583,C
443,0,3,male,25.0,0,0,7.0500,S


In [31]:
test = test.reset_index(drop=True)
test

,pclass,sex,age,sibsp,parch,fare,embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,3,male,NaN,0,0,8.4583,Q
4,1,male,54.0,0,0,51.8625,S
...,...,...,...,...,...,...,...
441,3,female,39.0,0,5,29.1250,Q
442,2,male,27.0,0,0,13.0000,S
443,1,female,19.0,0,0,30.0000,S
444,1,male,26.0,0,0,30.0000,C


In [32]:
#データの確認と前処理　テストデータ
print(train.shape)
train.head()

(445, 8)


,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,1,1,female,35.0,1,0,53.1000,S
1,0,3,male,35.0,0,0,8.0500,S
2,0,3,male,2.0,3,1,21.0750,S
3,1,2,female,14.0,1,0,30.0708,C
4,1,1,female,58.0,0,0,26.5500,S


In [33]:
#説明変数と目的変数に分ける
train_x = train.drop('survived',axis = 1)
train_y = train.iloc[:,0]
print(train_x.shape)
print(train_y.shape)

(445, 7)
(445,)


In [34]:
#trainデータ：one-hot-encoding
train_ohe = pd.get_dummies(train_x,dummy_na=True)
train_ohe.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,sex_nan,embarked_C,embarked_Q,embarked_S,embarked_nan
0,1,35.0,1,0,53.1000,1,0,0,0,0,1,0
1,3,35.0,0,0,8.0500,0,1,0,0,0,1,0
2,3,2.0,3,1,21.0750,0,1,0,0,0,1,0
3,2,14.0,1,0,30.0708,1,0,0,1,0,0,0
4,1,58.0,0,0,26.5500,1,0,0,0,0,1,0


In [35]:
#trainデータ：欠損値の補完
#インピュータークラスのインスタンス化と（列平均）の学習
imp = SimpleImputer()
imp.fit(train_ohe)

#学習済みimputerの適用：各列欠損値の置換
train_fin = pd.DataFrame(imp.transform(train_ohe) , columns = train_ohe.columns.values)

#結果表示
display(train_fin.iloc[10:20,1])

10    38.000000
11    29.211583
12    29.211583
13    29.211583
14    29.211583
15    66.000000
16    29.211583
17    27.000000
18    29.211583
19     3.000000
Name: age, dtype: float64

In [36]:
#family sizeのみ追加
train_fin['FamilySize'] = train_fin['sibsp'] + train_fin['parch'] + 1
train_fin['IsAlone'] = 0
train_fin.loc[train_fin['FamilySize'] == 1 , 'IsAlone']  = 1
train_fin.iloc[30:40,-1]

30    0
31    1
32    1
33    0
34    1
35    1
36    1
37    1
38    0
39    1
Name: IsAlone, dtype: int64

In [37]:
#testデータ：one_hot_encoding
test_ohe = pd.get_dummies(test,dummy_na=True)
test_ohe.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,sex_nan,embarked_C,embarked_Q,embarked_S,embarked_nan
0,3,22.0,1,0,7.2500,0,1,0,0,0,1,0
1,1,38.0,1,0,71.2833,1,0,0,1,0,0,0
2,3,26.0,0,0,7.9250,1,0,0,0,0,1,0
3,3,NaN,0,0,8.4583,0,1,0,0,1,0,0
4,1,54.0,0,0,51.8625,0,1,0,0,0,1,0


In [38]:
#testデータ：欠損値の補完
#インピュータークラスのインスタンス化と（列平均）の学習
imp = SimpleImputer()
imp.fit(test_ohe)

#学習済みimputerの適用：各列欠損値の置換
test_fin = pd.DataFrame(imp.transform(test_ohe) , columns = test_ohe.columns.values)

#結果表示
display(test_fin.iloc[10:20,1])

10    30.194915
11    31.000000
12    35.000000
13    34.000000
14    15.000000
15    19.000000
16    40.000000
17    30.194915
18    28.000000
19    42.000000
Name: age, dtype: float64

In [39]:
#特徴量追加
test_fin['FamilySize'] = test_fin['sibsp'] + test_fin['parch'] + 1
test_fin['IsAlone'] = 0
test_fin.loc[test_fin['FamilySize'] == 1 , 'IsAlone']  = 1
test_fin.iloc[30:40,-1]

30    0
31    0
32    1
33    1
34    0
35    1
36    0
37    1
38    0
39    0
Name: IsAlone, dtype: int64

In [40]:
#columnsのデータ項目を確認
cols_train = set(train_fin.columns.values)
cols_test = set(test_fin.columns.values)

#trainにあってtestにないデータ項目
diff1 = cols_train -cols_test
print('trainのみ：%s' % diff1)

#testにあってtrainにないデータ項目
diff2 = cols_test - cols_train
print('testのみ：%s' % diff2)

trainのみ：set()
testのみ：set()


#パラメータ探索
def objective(trial):
    param = {
    'objective':'binary',
    'max_bin': trial.suggest_int('max_bin',255,500) ,
    'learning_rate': 0.001,
    'num_leaves': trial.suggest_int('num_leaves',32,128)}
    
    #XGBoostClassifierのインスタンス作成
    lgb_train = lgb.Dataset(x_train,y_train)
    lgb_eval = lgb.Dataset(x_valid,y_valid, reference=lgb_train)

    #モデルフィッティング
    model = lgb.train(param,
                      lgb_train,
                      valid_sets=[lgb_train,lgb_eval],
                      verbose_eval=10,
                      num_boost_round=1000,
                      early_stopping_rounds=10
                     )

    y_pred_valid = model.predict(x_valid,num_iteration=model.best_iteration)
    score = log_loss(y_valid , y_pred_valid)
    return score


study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=1))
study.optimize(objective,n_trials=40)
study.best_params

In [41]:
print(train_fin.shape)
print(train_y.shape)

(445, 14)
(445,)


In [42]:
train_y[3]

1

In [47]:
#交差検証法の実装
y_preds = []
models = []
oof_train = np.zeros((len(train_fin),))
cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=1)

params = {
    'objective':'binary',
    'max_bin': 357 ,
    'learning_rate': 0.001,
    'num_leaves': 101
}

#StratifiedKFldを利用する事で正例と負例の割合を均一に保つ
for fold_id , (train_index ,valid_index) in enumerate(cv.split(train_fin,train_y)):
    x_tr = train_fin.loc[train_index, :]
    x_val = train_fin.loc[valid_index, :]
    y_tr = train_y[train_index]
    y_val = train_y[valid_index]
    
    lgb_train = lgb.Dataset(x_tr,y_tr)
    lgb_eval = lgb.Dataset(x_val,y_val,
                          reference=lgb_train)
    
    model = lgb.train(params,lgb_train,
                     valid_sets=[lgb_train,lgb_eval],
                     verbose_eval=10,
                      num_boost_round=1000,
                     early_stopping_rounds=10)
    
    oof_train[valid_index] = model.predict(x_val,num_iteration=model.best_iteration)
    y_pred = model.predict(test_fin,num_iteration=model.best_iteration)
    
    y_preds.append(y_pred)
    models.append(model)

/Users/saitoyuma/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/saitoyuma/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 143, number of negative: 213
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004607 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 164
[LightGBM] [Info] Number of data points in the train set: 356, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.401685 -> initscore=-0.398448
[LightGBM] [Info] Start training from score -0.398448
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	training's binary_logloss: 0.635902	valid_1's binary_logloss: 0.637131
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's binary_logloss: 0.591264	valid_1's binary_logloss: 0.591719
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[370]	training's binary_logloss: 0.562878	valid_1's binary_logloss: 0.563068
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[360]	training's binary_logloss: 0.555598	valid_1's binary_logloss: 0.575854
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[510]	training's binary_logloss: 0.523738	valid_1's binary_logloss: 0.553495
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[650]	training's binary_logloss: 0.497819	valid_1's binary_logloss: 0.54064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[760]	training's binary_logloss: 0.479457	valid_1's binary_logloss: 0.53338
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[770]	training's binary_logloss: 0.477924	valid_1's binary_logloss: 0.532829
[LightGBM] [Warning] N

[880]	training's binary_logloss: 0.462706	valid_1's binary_logloss: 0.527569
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[890]	training's binary_logloss: 0.461442	valid_1's binary_logloss: 0.527135
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] 

In [48]:
scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.4747349192688233, 0.49839142060163505, 0.48202954600856535, 0.49815527277434113, 0.5235159121141884]
0.49536541415351065


In [53]:
y_preds[0]

array([0.16148024, 0.78257918, 0.51382929, 0.33266366, 0.476259  ,
       0.43473274, 0.38472533, 0.19229033, 0.51785077, 0.66268578,
       0.31772435, 0.3809209 , 0.18543091, 0.22333822, 0.55562387,
       0.43601714, 0.41999877, 0.75167726, 0.43601714, 0.476259  ,
       0.1690397 , 0.38472533, 0.55562387, 0.43473274, 0.32246416,
       0.20760048, 0.28763458, 0.38472533, 0.47091458, 0.1690397 ,
       0.78257918, 0.476259  , 0.37364392, 0.28003729, 0.476259  ,
       0.38105202, 0.34098484, 0.66268578, 0.51785077, 0.38472533,
       0.31332294, 0.16096273, 0.25827974, 0.32246416, 0.47091458,
       0.43473274, 0.31064553, 0.3809209 , 0.16838872, 0.31772048,
       0.476259  , 0.41233792, 0.43601714, 0.51382929, 0.25827974,
       0.43601714, 0.22959841, 0.19314699, 0.26892454, 0.2432576 ,
       0.22916929, 0.49606106, 0.43869377, 0.22510311, 0.43601714,
       0.31244848, 0.476259  , 0.16096273, 0.3809209 , 0.66268578,
       0.49606106, 0.51785077, 0.38472533, 0.18512697, 0.20386

In [54]:
#提出用関数
sample[1] = y_preds[0]
sample.to_csv(f'submit/lightgbm3.tsv',sep='\t', header=None)